In [1]:
#required packages
!pip install pandas numpy scikit-learn matplotlib seaborn


In [2]:
import pandas as pd
import numpy as np
import requests
import time
from datetime import datetime, timedelta
import json
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

class CompoundRiskScorer:
    def __init__(self, etherscan_api_key: str):
        self.etherscan_api_key = etherscan_api_key
        self.base_url = "https://api.etherscan.io/api"

        self.compound_v2_contracts = {
            '0x3d9819210a31b4692a39c5d5c2e54a85a12e6b34': 'Comptroller',
            '0x5d3a536e4d6dbd6114cc1ead35777bab948e3643': 'cDAI',
            '0x4ddc2d193948926d02f9b1fe9e1daa0718270ed5': 'cETH',
            '0xf650c3d88d12db855b8bf7d11be6c55a4e07dcc9': 'cUSDT',
            '0x39aa39c021dfbabadb532e4a5b23baf35f6fe0a3': 'cUSDC',
            '0xc11b1268c1a384e55c48c2391d8d480264a3a7f4': 'cWBTC',
            '0x70e36f6bf80a52b3b46b3af8e106cc0ed743e8e4': 'cLEND',
            '0x35a18000230da775cac24873d00ff85bccded550': 'cUNI',
            '0x158079ee67fce2f58472a96584a73c7ab9ac95c1': 'cREP',
            '0xface851a4921ce59e912d19329929ce6da6eb0c7': 'cLINK',
            '0x95b4ef2869ebd94beb4eee400a99824bf5dc325b': 'cMKR'
        }

        self.compound_v3_contracts = {
            '0xc3d688b66703497daa19c4d3cacd6a6f5fe4b6b7': 'cUSDCv3',
            '0xa17581a9e3356d9a858b789d68b4d866e593ae94': 'cETHv3'
        }

        self.risk_weights = {
            'liquidation_risk': 0.30,
            'volatility_risk': 0.20,
            'concentration_risk': 0.15,
            'activity_risk': 0.15,
            'leverage_risk': 0.12,
            'recency_risk': 0.08
        }

        self.risk_thresholds = {
            'very_low': 150,
            'low': 300,
            'medium': 500,
            'high': 700,
            'very_high': 850
        }

    def fetch_wallet_transactions(self, wallet_address: str, max_retries: int = 3) -> List[Dict]:
        all_transactions = []

        for attempt in range(max_retries):
            try:
                params = {
                    'module': 'account',
                    'action': 'txlist',
                    'address': wallet_address,
                    'startblock': 0,
                    'endblock': 99999999,
                    'sort': 'desc',
                    'apikey': self.etherscan_api_key
                }

                response = requests.get(self.base_url, params=params, timeout=30)
                response.raise_for_status()
                data = response.json()

                if data['status'] == '1':
                    all_transactions.extend(data['result'])

                params['action'] = 'txlistinternal'
                response = requests.get(self.base_url, params=params, timeout=30)
                if response.status_code == 200:
                    internal_data = response.json()
                    if internal_data['status'] == '1':
                        for tx in internal_data['result']:
                            tx['is_internal'] = True
                        all_transactions.extend(internal_data['result'])

                params['action'] = 'tokentx'
                response = requests.get(self.base_url, params=params, timeout=30)
                if response.status_code == 200:
                    token_data = response.json()
                    if token_data['status'] == '1':
                        for tx in token_data['result']:
                            tx['is_token'] = True
                        all_transactions.extend(token_data['result'])

                time.sleep(0.25)
                break

            except Exception as e:
                print(f"Attempt {attempt + 1} failed for {wallet_address}: {str(e)}")
                if attempt < max_retries - 1:
                    time.sleep(2 ** attempt)
                else:
                    print(f"Failed to fetch data for {wallet_address} after {max_retries} attempts")

        return all_transactions

    def filter_compound_transactions(self, transactions: List[Dict]) -> List[Dict]:
        compound_addresses = set()
        compound_addresses.update([addr.lower() for addr in self.compound_v2_contracts.keys()])
        compound_addresses.update([addr.lower() for addr in self.compound_v3_contracts.keys()])

        compound_txns = []
        for tx in transactions:
            to_address = tx.get('to', '').lower()
            from_address = tx.get('from', '').lower()
            contract_address = tx.get('contractAddress', '').lower()

            if (to_address in compound_addresses or
                from_address in compound_addresses or
                contract_address in compound_addresses):
                compound_txns.append(tx)

        return compound_txns

    def extract_advanced_features(self, wallet_address: str, transactions: List[Dict]) -> Dict:
        if not transactions:
            return self.get_default_features()

        processed_txns = []
        for tx in transactions:
            try:
                processed_tx = {
                    'hash': tx.get('hash', ''),
                    'timestamp': int(tx.get('timeStamp', 0)),
                    'value': max(0, int(tx.get('value', 0)) / 1e18),
                    'gas_used': int(tx.get('gasUsed', 0)),
                    'gas_price': int(tx.get('gasPrice', 0)) / 1e9,
                    'is_error': tx.get('isError') == '1',
                    'to_address': tx.get('to', '').lower(),
                    'from_address': tx.get('from', '').lower(),
                    'is_internal': tx.get('is_internal', False),
                    'is_token': tx.get('is_token', False),
                    'token_symbol': tx.get('tokenSymbol', ''),
                    'token_value': max(0, int(tx.get('value', 0)) / (10 ** int(tx.get('tokenDecimal', 18))))
                }
                processed_txns.append(processed_tx)
            except (ValueError, TypeError):
                continue

        if not processed_txns:
            return self.get_default_features()

        processed_txns.sort(key=lambda x: x['timestamp'], reverse=True)

        now = datetime.now().timestamp()
        timestamps = [tx['timestamp'] for tx in processed_txns]
        first_tx_time = min(timestamps)
        last_tx_time = max(timestamps)

        total_transactions = len(processed_txns)
        account_age_days = max(1, (now - first_tx_time) / 86400)
        days_since_last_tx = (now - last_tx_time) / 86400

        successful_txns = [tx for tx in processed_txns if not tx['is_error']]
        failed_txns = [tx for tx in processed_txns if tx['is_error']]
        failure_rate = len(failed_txns) / total_transactions if total_transactions > 0 else 0

        eth_values = [tx['value'] for tx in successful_txns if tx['value'] > 0]
        total_eth_volume = sum(eth_values)
        avg_tx_value = np.mean(eth_values) if eth_values else 0
        median_tx_value = np.median(eth_values) if eth_values else 0
        value_std = np.std(eth_values) if len(eth_values) > 1 else 0
        value_volatility = value_std / max(avg_tx_value, 0.001)

        gas_costs = [tx['gas_used'] * tx['gas_price'] for tx in processed_txns]
        total_gas_cost = sum(gas_costs) / 1e9
        avg_gas_used = np.mean([tx['gas_used'] for tx in processed_txns])
        high_gas_txns = len([tx for tx in processed_txns if tx['gas_used'] > 300000])

        tx_frequency = total_transactions / account_age_days

        time_periods = {
            'last_7d': 7 * 86400,
            'last_30d': 30 * 86400,
            'last_90d': 90 * 86400
        }

        activity_metrics = {}
        for period, seconds in time_periods.items():
            period_txns = [tx for tx in processed_txns if (now - tx['timestamp']) <= seconds]
            activity_metrics[f'{period}_count'] = len(period_txns)
            activity_metrics[f'{period}_volume'] = sum(tx['value'] for tx in period_txns)

        unique_contracts = len(set(tx['to_address'] for tx in processed_txns if tx['to_address']))
        v2_interactions = len([tx for tx in processed_txns if tx['to_address'] in [addr.lower() for addr in self.compound_v2_contracts.keys()]])
        v3_interactions = len([tx for tx in processed_txns if tx['to_address'] in [addr.lower() for addr in self.compound_v3_contracts.keys()]])

        token_txns = [tx for tx in processed_txns if tx['is_token']]
        unique_tokens = len(set(tx['token_symbol'] for tx in token_txns if tx['token_symbol']))

        large_tx_count = len([tx for tx in processed_txns if tx['value'] > 10])
        rapid_tx_periods = self.detect_rapid_trading_periods(processed_txns)

        features = {
            'total_transactions': total_transactions,
            'account_age_days': account_age_days,
            'days_since_last_tx': days_since_last_tx,
            'tx_frequency': tx_frequency,
            'total_eth_volume': total_eth_volume,
            'avg_tx_value': avg_tx_value,
            'median_tx_value': median_tx_value,
            'value_volatility': value_volatility,
            'total_gas_cost': total_gas_cost,
            'avg_gas_used': avg_gas_used,
            'high_gas_tx_ratio': high_gas_txns / max(total_transactions, 1),
            'failure_rate': failure_rate,
            'failed_tx_count': len(failed_txns),
            **activity_metrics,
            'recent_activity_ratio': activity_metrics['last_30d_count'] / max(total_transactions, 1),
            'unique_contracts': unique_contracts,
            'contract_diversity_ratio': unique_contracts / max(total_transactions, 1),
            'v2_interaction_ratio': v2_interactions / max(total_transactions, 1),
            'v3_interaction_ratio': v3_interactions / max(total_transactions, 1),
            'unique_tokens': unique_tokens,
            'token_tx_ratio': len(token_txns) / max(total_transactions, 1),
            'large_tx_count': large_tx_count,
            'large_tx_ratio': large_tx_count / max(total_transactions, 1),
            'rapid_trading_periods': rapid_tx_periods,
            'volume_concentration': self.calculate_volume_concentration(processed_txns),
            'time_concentration': self.calculate_time_concentration(processed_txns)
        }

        return features

    def detect_rapid_trading_periods(self, transactions: List[Dict]) -> int:
        if len(transactions) < 5:
            return 0

        rapid_periods = 0
        sorted_txns = sorted(transactions, key=lambda x: x['timestamp'])

        for i in range(len(sorted_txns) - 4):
            time_span = sorted_txns[i + 4]['timestamp'] - sorted_txns[i]['timestamp']
            if time_span <= 3600:
                rapid_periods += 1

        return rapid_periods

    def calculate_volume_concentration(self, transactions: List[Dict]) -> float:
        values = [tx['value'] for tx in transactions if tx['value'] > 0]
        if not values:
            return 0

        total_volume = sum(values)
        if total_volume == 0:
            return 0

        shares = [(val / total_volume) ** 2 for val in values]
        return sum(shares)

    def calculate_time_concentration(self, transactions: List[Dict]) -> float:
        if len(transactions) < 2:
            return 0

        timestamps = [tx['timestamp'] for tx in transactions]
        time_diffs = np.diff(sorted(timestamps))

        if len(time_diffs) == 0:
            return 1.0

        mean_diff = np.mean(time_diffs)
        std_diff = np.std(time_diffs)

        if mean_diff == 0:
            return 1.0

        return min(std_diff / mean_diff, 10.0)

    def get_default_features(self) -> Dict:
        return {
            'total_transactions': 0,
            'account_age_days': 0,
            'days_since_last_tx': 999999,
            'tx_frequency': 0,
            'total_eth_volume': 0,
            'avg_tx_value': 0,
            'median_tx_value': 0,
            'value_volatility': 0,
            'total_gas_cost': 0,
            'avg_gas_used': 0,
            'high_gas_tx_ratio': 0,
            'failure_rate': 0,
            'failed_tx_count': 0,
            'last_7d_count': 0,
            'last_7d_volume': 0,
            'last_30d_count': 0,
            'last_30d_volume': 0,
            'last_90d_count': 0,
            'last_90d_volume': 0,
            'recent_activity_ratio': 0,
            'unique_contracts': 0,
            'contract_diversity_ratio': 0,
            'v2_interaction_ratio': 0,
            'v3_interaction_ratio': 0,
            'unique_tokens': 0,
            'token_tx_ratio': 0,
            'large_tx_count': 0,
            'large_tx_ratio': 0,
            'rapid_trading_periods': 0,
            'volume_concentration': 0,
            'time_concentration': 0
        }

    def calculate_enhanced_risk_components(self, features: Dict) -> Dict:
        liquidation_risk = min(100, (
            features['failure_rate'] * 80 +
            min(features['value_volatility'] * 25, 40) +
            (features['large_tx_ratio'] > 0.3) * 30 +
            (features['days_since_last_tx'] > 90) * 15 +
            (features['volume_concentration'] > 0.8) * 25
        ))

        volatility_risk = min(100, (
            min(features['value_volatility'] * 20, 50) +
            (features['rapid_trading_periods'] > 2) * 30 +
            min(features['tx_frequency'] * 15, 25) +
            (features['time_concentration'] > 5) * 20 +
            (features['recent_activity_ratio'] > 0.7) * 15
        ))

        concentration_risk = min(100, (
            (1 / max(features['contract_diversity_ratio'], 0.01)) * 20 +
            features['volume_concentration'] * 40 +
            (features['unique_tokens'] < 3) * 25 +
            (features['total_eth_volume'] > 50) * 15
        ))

        activity_risk = min(100, (
            (features['total_transactions'] == 0) * 90 +
            (features['tx_frequency'] < 0.01) * 50 +
            (features['account_age_days'] < 30) * 25 +
            (features['last_30d_count'] == 0) * 35
        ))

        leverage_risk = min(100, (
            features['high_gas_tx_ratio'] * 60 +
            (features['total_gas_cost'] > 2) * 25 +
            features['failure_rate'] * 40 +
            (features['v3_interaction_ratio'] > 0.5) * 15
        ))

        recency_risk = min(100, (
            min(features['days_since_last_tx'] / 30, 1) * 70 +
            (features['recent_activity_ratio'] < 0.1) * 25 +
            (features['last_7d_count'] == 0) * 15
        ))

        return {
            'liquidation_risk': liquidation_risk,
            'volatility_risk': volatility_risk,
            'concentration_risk': concentration_risk,
            'activity_risk': activity_risk,
            'leverage_risk': leverage_risk,
            'recency_risk': recency_risk
        }

    def calculate_final_score(self, risk_components: Dict, features: Dict) -> int:
        weighted_score = sum(
            risk_components[component] * self.risk_weights[component]
            for component in risk_components
        )

        modifiers = 0

        if features['total_transactions'] == 0:
            modifiers += 10

        if (features['total_transactions'] > 10 and
            features['failure_rate'] < 0.05 and
            features['days_since_last_tx'] < 30):
            modifiers -= 15

        if (features['failure_rate'] > 0.1 and
            features['value_volatility'] > 3):
            modifiers += 20

        adjusted_score = max(0, weighted_score + modifiers)

        if adjusted_score < 20:
            final_score = int(adjusted_score * 5)
        elif adjusted_score < 35:
            final_score = int(100 + (adjusted_score - 20) * 8)
        elif adjusted_score < 50:
            final_score = int(220 + (adjusted_score - 35) * 12)
        elif adjusted_score < 65:
            final_score = int(400 + (adjusted_score - 50) * 16)
        elif adjusted_score < 80:
            final_score = int(640 + (adjusted_score - 65) * 20)
        else:
            final_score = int(940 + min((adjusted_score - 80) * 3, 60))

        return min(max(final_score, 0), 1000)

    def analyze_wallet(self, wallet_address: str) -> Tuple[int, Dict, Dict]:
        print(f"Analyzing wallet: {wallet_address}")

        try:
            all_transactions = self.fetch_wallet_transactions(wallet_address)
            compound_txns = self.filter_compound_transactions(all_transactions)
            features = self.extract_advanced_features(wallet_address, compound_txns)
            risk_components = self.calculate_enhanced_risk_components(features)
            final_score = self.calculate_final_score(risk_components, features)

            return final_score, features, risk_components

        except Exception as e:
            print(f"Error analyzing wallet {wallet_address}: {str(e)}")
            default_features = self.get_default_features()
            default_risks = {key: 50 for key in self.risk_weights.keys()}
            return 500, default_features, default_risks

    def analyze_wallet_list(self, wallet_addresses: List[str]) -> pd.DataFrame:
        results = []

        print(f"Starting analysis of {len(wallet_addresses)} wallets...")

        for i, wallet in enumerate(wallet_addresses, 1):
            try:
                print(f"Processing wallet {i}/{len(wallet_addresses)}: {wallet}")

                score, features, risk_components = self.analyze_wallet(wallet)

                result = {
                    'wallet_id': wallet,
                    'score': score,
                    **features,
                    **risk_components
                }
                results.append(result)

                if i % 10 == 0:
                    print(f"Completed {i}/{len(wallet_addresses)} wallets")
                    print(f"Recent scores: {[r['score'] for r in results[-5:]]}")

            except Exception as e:
                print(f"Critical error analyzing wallet {wallet}: {str(e)}")
                default_result = {
                    'wallet_id': wallet,
                    'score': 400,
                    **self.get_default_features(),
                    **{key: 60 for key in self.risk_weights.keys()}
                }
                results.append(default_result)

        return pd.DataFrame(results)

    def generate_risk_report(self, results_df: pd.DataFrame) -> Dict:
        scores = results_df['score']

        report = {
            'total_wallets': len(results_df),
            'score_stats': {
                'mean': scores.mean(),
                'median': scores.median(),
                'std': scores.std(),
                'min': scores.min(),
                'max': scores.max(),
                'q25': scores.quantile(0.25),
                'q75': scores.quantile(0.75)
            },
            'risk_categories': {
                'very_low': len(scores[scores < self.risk_thresholds['very_low']]),
                'low': len(scores[(scores >= self.risk_thresholds['very_low']) &
                               (scores < self.risk_thresholds['low'])]),
                'medium': len(scores[(scores >= self.risk_thresholds['low']) &
                                  (scores < self.risk_thresholds['medium'])]),
                'high': len(scores[(scores >= self.risk_thresholds['medium']) &
                                (scores < self.risk_thresholds['high'])]),
                'very_high': len(scores[scores >= self.risk_thresholds['high']])
            }
        }

        report['top_risk_wallets'] = results_df.nlargest(10, 'score')[['wallet_id', 'score']].to_dict('records')
        report['lowest_risk_wallets'] = results_df.nsmallest(10, 'score')[['wallet_id', 'score']].to_dict('records')

        return report

def main():
    ETHERSCAN_API_KEY = "P389WNCA91Z525BHD8DIKDXJ1QM281W1IS"

    try:
        wallet_df = pd.read_csv('Wallet id - Sheet1.csv')
        wallet_addresses = wallet_df['wallet_id'].tolist()
        print(f"Loaded {len(wallet_addresses)} wallet addresses from CSV")
    except FileNotFoundError:
        print("CSV file not found. Please ensure 'Wallet id - Sheet1.csv' is in the current directory.")
        return
    except Exception as e:
        print(f"Error loading CSV: {str(e)}")
        return

    scorer = CompoundRiskScorer(ETHERSCAN_API_KEY)

    print("Starting comprehensive wallet risk analysis...")
    results_df = scorer.analyze_wallet_list(wallet_addresses)

    print("\nSaving results...")

    submission_df = results_df[['wallet_id', 'score']].copy()
    submission_df.to_csv('wallet_risk_scores.csv', index=False)

    results_df.to_csv('detailed_wallet_analysis.csv', index=False)

    risk_report = scorer.generate_risk_report(results_df)

    with open('risk_analysis_report.json', 'w') as f:
        json.dump(risk_report, f, indent=2, default=str)

    print("COMPOUND WALLET RISK ANALYSIS COMPLETE")
    print("\n")
    print(f"Total Wallets Analyzed: {risk_report['total_wallets']}")
    print(f"\nScore Statistics:")
    print(f"  Average Score: {risk_report['score_stats']['mean']:.1f}")
    print(f"  Median Score:  {risk_report['score_stats']['median']:.1f}")
    print(f"  Score Range:   {risk_report['score_stats']['min']:.0f} - {risk_report['score_stats']['max']:.0f}")
    print(f"  Std Deviation: {risk_report['score_stats']['std']:.1f}")

    print(f"\nRisk Categories:")
    for category, count in risk_report['risk_categories'].items():
        percentage = (count / risk_report['total_wallets']) * 100
        print(f"  {category.replace('_', ' ').title():12}: {count:3d} wallets ({percentage:5.1f}%)")

    print(f"\nFiles Generated:")
    print(f"  - wallet_risk_scores.csv (submission format)")
    print(f"  - detailed_wallet_analysis.csv (comprehensive analysis)")
    print(f"  - risk_analysis_report.json (summary report)")

    print(f"\nTop 5 Highest Risk Wallets:")
    for i, wallet in enumerate(risk_report['top_risk_wallets'][:5], 1):
        print(f"  {i}. {wallet['wallet_id']} (Score: {wallet['score']})")

    print(f"\nTop 5 Lowest Risk Wallets:")
    for i, wallet in enumerate(risk_report['lowest_risk_wallets'][:5], 1):
        print(f"  {i}. {wallet['wallet_id']} (Score: {wallet['score']})")


if __name__ == "__main__":
    main()

Loaded 103 wallet addresses from CSV
Starting comprehensive wallet risk analysis...
Starting analysis of 103 wallets...
Processing wallet 1/103: 0x0039f22efb07a647557c7c5d17854cfd6d489ef3
Analyzing wallet: 0x0039f22efb07a647557c7c5d17854cfd6d489ef3
Processing wallet 2/103: 0x06b51c6882b27cb05e712185531c1f74996dd988
Analyzing wallet: 0x06b51c6882b27cb05e712185531c1f74996dd988
Processing wallet 3/103: 0x0795732aacc448030ef374374eaae57d2965c16c
Analyzing wallet: 0x0795732aacc448030ef374374eaae57d2965c16c
Processing wallet 4/103: 0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
Analyzing wallet: 0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
Processing wallet 5/103: 0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae
Analyzing wallet: 0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae
Processing wallet 6/103: 0x104ae61d8d487ad689969a17807ddc338b445416
Analyzing wallet: 0x104ae61d8d487ad689969a17807ddc338b445416
Processing wallet 7/103: 0x111c7208a7e2af345d36b6d4aace8740d61a3078
Analyzing wallet: 0x111c7208a7e2af345d